### Build a Question Answering application over a Graph Database

In [81]:
import os
from dotenv import load_dotenv
load_dotenv()

openai_api_key=os.getenv("OPENAI_API_KEY")
NEO4J_URI=os.getenv("NEO4J_URI")
NEO4J_USERNAME=os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD= os.getenv("NEO4J_PASSWORD")

In [26]:
from langchain_community.graphs import Neo4jGraph
graph=Neo4jGraph(url=NEO4J_URI,username=NEO4J_USERNAME,password=NEO4J_PASSWORD)
graph

In [174]:
## Dataset Moview
moview_query="""
LOAD CSV WITH HEADERS FROM
'https://raw.githubusercontent.com/ashishkumarsahani/vettura-genai/refs/heads/main/Codes/Week_3/Day_1/graph-dataset/movies_small.csv' as row

MERGE (m:Movie {movieId: toInteger(row.movieId)})
SET m.title = row.title,
    m.released = date(row.released),
    m.imdbRating = toFloat(row.imdbRating),
    m.genre = row.genres

FOREACH (director IN split(row.director, '|') |
    MERGE (p:Person {name: trim(director)})
    MERGE (p)-[:DIRECTED]->(m)
)

FOREACH (actor IN split(row.actors, '|') |
    MERGE (p:Person {name: trim(actor)})
    MERGE (p)-[:ACTED_IN]->(m)
)

FOREACH (genre IN split(row.genres, '|') |
    MERGE (g:Genre {name: trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g)
)


"""

In [175]:
print(moview_query)


LOAD CSV WITH HEADERS FROM
'https://raw.githubusercontent.com/ashishkumarsahani/vettura-genai/refs/heads/main/Codes/Week_3/Day_1/graph-dataset/movies_small.csv' as row

MERGE (m:Movie {movieId: toInteger(row.movieId)}) 
SET m.title = row.title,
    m.released = date(row.released),
    m.imdbRating = toFloat(row.imdbRating),
    m.genre = row.genres

FOREACH (director IN split(row.director, '|') |
    MERGE (p:Person {name: trim(director)})
    MERGE (p)-[:DIRECTED]->(m)
)

FOREACH (actor IN split(row.actors, '|') |  
    MERGE (p:Person {name: trim(actor)})
    MERGE (p)-[:ACTED_IN]->(m)       
)

FOREACH (genre IN split(row.genres, '|') |
    MERGE (g:Genre {name: trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g)
)





In [176]:
graph.query(moview_query)

[]

In [177]:
graph.refresh_schema()
print(graph.schema)

Node properties:
Person {name: STRING}
Movie {id: STRING, released: DATE, title: STRING, imdbRating: FLOAT, movieId: INTEGER, genre: STRING}
Genre {name: STRING}
Relationship properties:

The relationships:
(:Person)-[:DIRECTED]->(:Movie)
(:Person)-[:ACTED_IN]->(:Movie)
(:Movie)-[:IN_GENRE]->(:Genre)


In [178]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(api_key=openai_api_key,model_name="gpt-4o-mini")
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7fc4409fc9d0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7fc441389950>, root_client=<openai.OpenAI object at 0x7fc44198e0d0>, root_async_client=<openai.AsyncOpenAI object at 0x7fc4433ba650>, model_name='gpt-4o-mini', model_kwargs={}, openai_api_key=SecretStr('**********'))

In [179]:
from langchain.chains import GraphCypherQAChain
chain=GraphCypherQAChain.from_llm(graph=graph,llm=llm,verbose=True, allow_dangerous_requests=True)
chain

GraphCypherQAChain(verbose=True, graph=<langchain_community.graphs.neo4j_graph.Neo4jGraph object at 0x7fc444ab5350>, cypher_generation_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['question', 'schema'], input_types={}, partial_variables={}, template='Task:Generate Cypher statement to query a graph database.\nInstructions:\nUse only the provided relationship types and properties in the schema.\nDo not use any other relationship types or properties that are not provided.\nSchema:\n{schema}\nNote: Do not include any explanations or apologies in your responses.\nDo not respond to any questions that might ask anything else than for you to construct a Cypher statement.\nDo not include any text except the generated Cypher statement.\n\nThe question is:\n{question}'), llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7fc4409fc9d0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7fc441389950>, root_client=<open

In [180]:
response=chain.invoke({"query":"Who was the director of the movie Casino"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (director:Person)-[:DIRECTED]->(movie:Movie {title: 'Casino'}) RETURN director.name
Full Context:
[{'director.name': 'Martin Scorsese'}, {'director.name': 'Martin Scorsese'}, {'director.name': 'Martin Scorsese'}]

> Finished chain.


{'query': 'Who was the director of the movie Casino',
 'result': 'The director of the movie Casino was Martin Scorsese.'}

In [181]:
response=chain.invoke({"query":"Who were the actors of the movie Casino"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:

MATCH (p:Person)-[:ACTED_IN]->(m:Movie {title: 'Casino'})
RETURN p.name

Full Context:
[{'p.name': 'Robert De Niro'}, {'p.name': 'Joe Pesci'}, {'p.name': 'Sharon Stone'}, {'p.name': 'James Woods'}, {'p.name': 'Robert De Niro'}, {'p.name': 'Joe Pesci'}, {'p.name': 'Sharon Stone'}, {'p.name': 'James Woods'}, {'p.name': 'Robert De Niro'}, {'p.name': 'Joe Pesci'}]

> Finished chain.


{'query': 'Who were the actors of the movie Casino',
 'result': 'The actors of the movie Casino include Robert De Niro, Joe Pesci, Sharon Stone, and James Woods.'}

In [182]:
response=chain.invoke({"query":"How many artists are there?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (p:Person) RETURN COUNT(p) AS artistCount

Full Context:
[{'artistCount': 1239}]

> Finished chain.


{'query': 'How many artists are there?', 'result': 'There are 1,239 artists.'}

In [183]:
response=chain.invoke({"query":"How many movies has Tom Hanks acted in"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (p:Person {name: 'Tom Hanks'})-[:ACTED_IN]->(m:Movie)
RETURN COUNT(m) AS moviesActedIn

Full Context:
[{'moviesActedIn': 6}]

> Finished chain.


{'query': 'How many movies has Tom Hanks acted in',
 'result': "I don't know the answer."}

In [223]:
examples = [
    {
        "question": "How many artists are there?",
        "query": "MATCH (a:Person)-[:ACTED_IN]->(:Movie) RETURN count(DISTINCT a)",
    },
    {
        "question": "Which actors played in the movie Casino?",
        "query": "MATCH (m:Movie {{title: 'Casino'}})<-[:ACTED_IN]-(a) RETURN a.name",
    },
    {
        "question": "List all the genres of the movie Schindler's List",
        "query": "MATCH (m:Movie {{title: 'Schindler\\'s List'}})-[:IN_GENRE]->(g:Genre) RETURN g.name",
    },
    {
        "question": "Which actors have worked in movies from both the comedy and action genres?",
        "query": "MATCH (a:Person)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g1:Genre), (a)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g2:Genre) WHERE g1.name = 'Comedy' AND g2.name = 'Action' RETURN DISTINCT a.name",
    },
    {
        "question": "Which directors have made movies with at least three different actors named 'John'?",
        "query": "MATCH (d:Person)-[:DIRECTED]->(m:Movie)<-[:ACTED_IN]-(a:Person) WHERE a.name STARTS WITH 'John' WITH d, COUNT(DISTINCT a) AS JohnsCount WHERE JohnsCount >= 3 RETURN d.name",
    },
    {
        "question": "Identify movies where directors also played a role in the film.",
        "query": "MATCH (p:Person)-[:DIRECTED]->(m:Movie), (p)-[:ACTED_IN]->(m) RETURN m.title, p.name",
    },
    {
        "question": "Find the actor with the highest number of movies in the database.",
        "query": "MATCH (a:Actor)-[:ACTED_IN]->(m:Movie) RETURN a.name, COUNT(m) AS movieCount ORDER BY movieCount DESC LIMIT 1",
    },
]

In [224]:
from langchain_core.prompts import FewShotPromptTemplate,PromptTemplate
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_neo4j import Neo4jVector
from langchain_openai import OpenAIEmbeddings

example_prompt=PromptTemplate.from_template(
    "User input:{question}\n Cypher query:{query}"
)

prompt=FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="You are a Neo4j expert. Given an input question,create a syntactically very accurate Cypher query.",
    suffix="User input: {question}\nCypher query: ",
    input_variables=["question"]
)

In [225]:
print(prompt.format(question="How many artists are there?"))

You are a Neo4j expert. Given an input question,create a syntactically very accurate Cypher query.

User input:How many artists are there?
 Cypher query:MATCH (a:Person)-[:ACTED_IN]->(:Movie) RETURN count(DISTINCT a)

User input:Which actors played in the movie Casino?
 Cypher query:MATCH (m:Movie {title: 'Casino'})<-[:ACTED_IN]-(a) RETURN a.name

User input:List all the genres of the movie Schindler's List
 Cypher query:MATCH (m:Movie {title: 'Schindler\'s List'})-[:IN_GENRE]->(g:Genre) RETURN g.name

User input:Which actors have worked in movies from both the comedy and action genres?
 Cypher query:MATCH (a:Person)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g1:Genre), (a)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g2:Genre) WHERE g1.name = 'Comedy' AND g2.name = 'Action' RETURN DISTINCT a.name

User input:Which directors have made movies with at least three different actors named 'John'?
 Cypher query:MATCH (d:Person)-[:DIRECTED]->(m:Movie)<-[:ACTED_IN]-(a:Person) WHERE a.name STARTS WITH 'John' W

In [226]:
res = chain.invoke(prompt.format(question="How many artists are there?"))



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (a:Person) RETURN count(DISTINCT a)
Full Context:
[{'count(DISTINCT a)': 1239}]

> Finished chain.


In [227]:
res

{'query': "You are a Neo4j expert. Given an input question,create a syntactically very accurate Cypher query.\n\nUser input:How many artists are there?\n Cypher query:MATCH (a:Person)-[:ACTED_IN]->(:Movie) RETURN count(DISTINCT a)\n\nUser input:Which actors played in the movie Casino?\n Cypher query:MATCH (m:Movie {title: 'Casino'})<-[:ACTED_IN]-(a) RETURN a.name\n\nUser input:List all the genres of the movie Schindler's List\n Cypher query:MATCH (m:Movie {title: 'Schindler\\'s List'})-[:IN_GENRE]->(g:Genre) RETURN g.name\n\nUser input:Which actors have worked in movies from both the comedy and action genres?\n Cypher query:MATCH (a:Person)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g1:Genre), (a)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g2:Genre) WHERE g1.name = 'Comedy' AND g2.name = 'Action' RETURN DISTINCT a.name\n\nUser input:Which directors have made movies with at least three different actors named 'John'?\n Cypher query:MATCH (d:Person)-[:DIRECTED]->(m:Movie)<-[:ACTED_IN]-(a:Person) WHERE 

In [231]:
graph.query('MATCH (a:Person) RETURN count(DISTINCT a) AS artistCount')

[{'artistCount': 1239}]